### Imports and Initializations

In [77]:
from torch.utils.data import DataLoader
import torch
import helper

In [78]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Initialize the Dataset for Training the Model

In [79]:
dataset = helper.HeartDataset("./Heart_disease_cleveland_new.csv")

In [80]:
# 80-20 train test split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create the dataloaders
train_dl = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=16, shuffle=True)

### Create the Classifier Model

In [81]:
model = helper.Classifier().to(device)

In [82]:
loss_function = torch.nn.MSELoss()
lr= 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-05)
epochs = 1000

In [83]:
for ep in range(epochs):
    for i, (batch, batch_labels) in enumerate(train_dl):
        model.train()
        batch = batch.to(device)
        batch_labels = batch_labels.unsqueeze(-1).to(device)

        output = model(batch)
        #print(output)
        #print(batch_labels.shape)
        #print(output.shape)
        
        loss = loss_function(output, batch_labels)

        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

    model.eval()
    for batch, batch_labels in test_dl:
        batch = batch.to(device)
        batch_labels = batch_labels.to(device)

        output = model(batch)
        #print(output)
        loss = loss_function(output, batch_labels)
    print('Epoch[{}/{}], loss: {:.6f}'.format(ep + 1, epochs, loss.data.item()))

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([13])) that is different to the input size (torch.Size([13, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch[1/1000], loss: 0.429262
Epoch[2/1000], loss: 0.208706
Epoch[3/1000], loss: 0.331590
Epoch[4/1000], loss: 0.319485
Epoch[5/1000], loss: 0.427250
Epoch[6/1000], loss: 0.455858
Epoch[7/1000], loss: 0.181792
Epoch[8/1000], loss: 0.312683
Epoch[9/1000], loss: 0.307786
Epoch[10/1000], loss: 0.251065
Epoch[11/1000], loss: 0.344861
Epoch[12/1000], loss: 0.293859
Epoch[13/1000], loss: 0.268165
Epoch[14/1000], loss: 0.246192
Epoch[15/1000], loss: 0.229008
Epoch[16/1000], loss: 0.277496
Epoch[17/1000], loss: 0.174908
Epoch[18/1000], loss: 0.314596
Epoch[19/1000], loss: 0.277014
Epoch[20/1000], loss: 0.318811
Epoch[21/1000], loss: 0.341918
Epoch[22/1000], loss: 0.362556
Epoch[23/1000], loss: 0.321903
Epoch[24/1000], loss: 0.340371
Epoch[25/1000], loss: 0.382421
Epoch[26/1000], loss: 0.378962
Epoch[27/1000], loss: 0.329385
Epoch[28/1000], loss: 0.356101
Epoch[29/1000], loss: 0.322101
Epoch[30/1000], loss: 0.307439
Epoch[31/1000], loss: 0.329004
Epoch[32/1000], loss: 0.344591
Epoch[33/1000], l

In [84]:
model.eval()
accuracy = 0
for i in range(len(dataset)):
    id = "Incorrect"
    data, label = dataset[i]

    data = data.to(device)
    label = label.to(device)
    
    output = model(data)
    if(int(output[0]) == label):
        accuracy += 1
        id = "Correct"
    print("Label: {}\t|Model Guess: {}\t| {}".format(label, int(output[0]), id))
print("Accuracy on overall dataset: " + str(accuracy/len(dataset)))

Label: 0.0	|Model Guess: 0	| Correct
Label: 1.0	|Model Guess: 1	| Correct
Label: 1.0	|Model Guess: 1	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 0.0	|Model Guess: 0	| Correct
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 1.0	|Model Guess: 1	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 1.0	|Model Guess: 1	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0

In [85]:

accuracy = 0
for i in range(len(test_dataset)):
    id = "Incorrect"
    data, label = test_dataset[i]

    data = data.to(device)
    label = label.to(device)
    
    output = model(data)
    if(int(output[0]) == label):
        accuracy += 1
        id = "Correct"
    print("Label: {}\t|Model Guess: {}\t| {}".format(label, int(output[0]), id))
print("Accuracy on Unseen(test) data: " + str(accuracy/len(test_dataset)))

Label: 0.0	|Model Guess: 0	| Correct
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 1.0	|Model Guess: 1	| Correct
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 0.0	|Model Guess: 0	| Correct
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 1.0	|Model Guess: 1	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 0.0	|Model Guess: 0	| Correct
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 0.0	|Model Guess: 0	| Correct
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 1.0	|Model Guess: 0	| Incorrect
Label: 1.0	|Mode